In [3]:
# finalize model andmake a prediction for monthly births with xgboost
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
#from xgboost import XGBRegressor

 


In [4]:
 
# transform a timeseries dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
       n_vars = 1 if type(data) is list else data.shape[1]
       df = DataFrame(data)
       cols = list()
       # input sequence (t-n, ... t-1)
       for i in range(n_in, 0, -1):
              cols.append(df.shift(i))
       # forecast sequence (t, t+1, ... t+n)
       for i in range(0, n_out):
              cols.append(df.shift(-i))
       # put it all together
       agg = concat(cols, axis=1)
       # drop rows with NaN values
       if dropnan:
              agg.dropna(inplace=True)
       return agg.values

In [14]:
# load the dataset
series =read_csv('./ft.csv', header=0, index_col=0,encoding='GB2312')
values = series.values
# transform the timeseries data into supervised learning
values.shape

(10713, 1)

In [16]:
values[:10]

array([[2387.489258],
       [2383.365479],
       [2388.100098],
       [2412.537109],
       [2453.316406],
       [2439.112305],
       [2417.118896],
       [2449.192383],
       [2507.688477],
       [2526.321533]])

In [10]:
train =series_to_supervised(values, n_in=3)
# split into input andoutput columns
trainX, trainy =train[:, :-1], train[:, -1]
trainX.shape,trainy.shape

((10710, 3), (10710,))

In [12]:
trainX[:5],trainy[:5]

(array([[2387.489258, 2383.365479, 2388.100098],
        [2383.365479, 2388.100098, 2412.537109],
        [2388.100098, 2412.537109, 2453.316406],
        [2412.537109, 2453.316406, 2439.112305],
        [2453.316406, 2439.112305, 2417.118896]]),
 array([2412.537109, 2453.316406, 2439.112305, 2417.118896, 2449.192383]))

# 训练模型

In [ ]:
# fit model
model =XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(trainX,trainy)


In [9]:
# construct an inputfor a new preduction
row = values[-3:].flatten()
# make a one-stepprediction
yhat =model.predict(asarray([row]))
print('Input: %s,Predicted: %.3f' % (row, yhat[0]))

Input: [3281.399902 3279.800049 3280.199951],Predicted: 3271.301
